In [26]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import json
import glob
import os
import re

In [2]:
paths = glob.glob("../exp/bidir_features/*.json")
data = [json.load(open(path)) for path in paths]

In [10]:
queries = pd.DataFrame.from_records([{
    **algo,
    'graph': { 'europe': 'DIMACs Europe', 'osm_ger': 'OSM Germany', 'osm_europe': 'OSM Europe' }[[x for x in run['args'][1].split('/') if x != ''][-1]],
    'experiment': exp['experiment'],
    'factor': exp['factor'],
    'probability': exp.get('probability', 1),
    'speed': exp.get('speed', 0),
    'potential': run['potential'],
    'num_queue_pops': algo['num_queue_pops'] + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo['num_queue_pushs'] + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo['num_relaxed_arcs'] + algo.get('core_search', {}).get('num_relaxed_arcs', 0),
    }
    for run in data for exp in run['experiments'] for algo in exp['algo_runs']])

queries['choose_direction_strategy'] = queries['choose_direction_strategy'].fillna('unidir')
queries['bidir_pot'] = queries['bidir_pot'].fillna('unidir')
queries['improved_pruning'] = queries['improved_pruning'].fillna('unidir')
queries['num_queue_pushs_k'] = queries['num_queue_pushs'] / 1000

In [4]:
pd.set_option('display.max_rows', 500)

In [11]:
queries = queries.loc[lambda x: x['graph'] == 'OSM Germany']

In [12]:
queries.groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'choose_direction_strategy', 'improved_pruning'])[['running_time_ms', 'num_queue_pushs_k']].mean().unstack(0)

running_time_ms  \
potential                                                                                                                                                  ALT   
experiment                   factor probability algo                                      bidir_pot choose_direction_strategy improved_pruning                   
probabilistic_scale_by_speed 1.25   0.00        Bidrectional Dijkstra Query               Average   alternating               False                     562.86   
                                                                                                                              True                      563.81   
                                                                                                    min_key                   False                     584.80   
                                                                                                                              True                      590.12   
                                                                                          Symmetric alternating               False                   1,777.17   
                                                                                                                              True                      573.14   
                                                                                                    min_key                   False                   1,761.24   
                                                                                                                              True                      605.35   
                                                Dijkstra Query                            unidir    unidir                    unidir                    711.28   
                                                Virtual Topocore Bidirectional Core Query Average   alternating               False                     158.72   
                                                                                                                              True                      160.36   
                                                                                                    min_key                   False                     165.85   
                                                                                                                              True                      167.22   
                                                                                          Symmetric alternating               False                     592.90   
                                                                                                                              True                      176.69   
                                                                                                    min_key                   False                     582.03   
                                                                                                                              True                      186.60   
                                                Virtual Topocore Component Query          unidir    unidir                    unidir                    216.35   
                                    1.00        Bidrectional Dijkstra Query               Average   alternating               False                      63.61   
                                                                                                                              True                       64.38   
                                                                                                    min_key                   False                      88.06   
                                                                                                                              True                       89.21   
                                                                                          Sym

### Effectiveness new pruning

In [30]:
pruning = queries.loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'improved_pruning'])[['running_time_ms', 'num_queue_pushs_k']].mean().unstack(0) \
    .loc[('weight_scale', 1.05, 1.00)] \
    .rename(columns={
        'running_time_ms': 'Running time [ms]',
        'num_queue_pushs_k': 'Queue pushs [$\\cdot 10^3$]',
        'CH': 'CH-Pot.',
        'CCH': 'CCH-Pot.',
    }) \
    .rename(index={
        'Average': 'Avg.',
        'Symmetric': 'Sym.',
    })
pruning

Running time [ms]  \
potential                                                                          ALT   
algo                                      bidir_pot improved_pruning                     
Bidrectional Dijkstra Query               Avg.      False                       130.99   
                                                    True                        132.76   
                                          Sym.      False                       797.74   
                                                    True                        270.95   
Virtual Topocore Bidirectional Core Query Avg.      False                        34.53   
                                                    True                         35.24   
                                          Sym.      False                       244.80   
                                                    True                         75.87   

                                                                              \
potential                                                            CH-Pot.   
algo                                      bidir_pot improved_pruning           
Bidrectional Dijkstra Query               Avg.      False              56.46   
                                                    True               57.97   
                                          Sym.      False             112.55   
                                                    True               53.85   
Virtual Topocore Bidirectional Core Query Avg.      False              17.85   
                                                    True               18.01   
                                          Sym.      False              38.05   
                                                    True               19.65   

                                                                             \
potential                                                            Oracle   
algo                                      bidir_pot improved_pruning          
Bidrectional Dijkstra Query               Avg.      False             33.90   
                                                    True              34.57   
                                          Sym.      False             81.50   
                                                    True              33.75   
Virtual Topocore Bidirectional Core Query Avg.      False              8.97   
                                                    True               9.18   
                                          Sym.      False             24.12   
                                                    True              10.37   

                                                                              \
potential                                                               Zero   
algo                                      bidir_pot improved_pruning           
Bidrectional Dijkstra Query               Avg.      False            1555.31   
                                                    True             1550.62   
                                          Sym.      False            6137.12   
                                                    True             1545.56   
Virtual Topocore Bidirectional Core Query Avg.      False             395.43   
                                                    True              399.60   
                                          Sym.      False            1641.51   
                                                    True              396.72   

                                                                     Queue pushs [$\cdot 10^3$]  \
potential                                                                                   ALT   
algo                                      bidir_pot improved_pruning                              
Bidrectional Dijkstra Query               Avg.      False                                307.35   
                                                  

In [31]:
lines = pruning.to_latex(escape=False).split("\n")

lines = [R'\begin{tabular}{lllrrrrrrrr}'] + [lines[1]] + [
    ' & '.join(["Low Deg.", "Bidir.", "Improved"] + lines[2].split(' & ')[3:]) + R" \cmidrule(lr){4-7} \cmidrule(lr){8-11}",
    ' & '.join(["Opt.",     "Pot.",   "Pruning" ] + lines[3].split(' & ')[3:])
] + lines[5:]
# linespace

output = "\n".join(lines) + "\n"
output = output.replace('Bidrectional Dijkstra Query', R'\multirow{4}{*}{\xmark}')
output = output.replace('Virtual Topocore Bidirectional Core Query', R'\multirow{4}{*}{\cmark}')
output = output.replace('True', '\\cmark')
output = output.replace('False', '\\xmark')
output = re.sub(re.compile('([0-9]{3}(?=[0-9]))'), '\\g<0>,\\\\', output[::-1])[::-1]

print(output)

\begin{tabular}{lllrrrrrrrr}
\toprule
Low Deg. & Bidir. & Improved & \multicolumn{4}{l}{Running time [ms]} & \multicolumn{4}{l}{Queue pushs [$\cdot 10^3$]} \\ \cmidrule(lr){4-7} \cmidrule(lr){8-11}
Opt. & Pot. & Pruning &               ALT & CH-Pot. & Oracle &    Zero &                        ALT & CH-Pot. & Oracle &     Zero \\
\midrule
\multirow{4}{*}{\xmark} & Avg. & \xmark &            130.99 &   56.46 &  33.90 & 1\,555.31 &                     307.35 &  116.33 & 116.33 &  4\,797.13 \\
                                          &      & \cmark &            132.76 &   57.97 &  34.57 & 1\,550.62 &                     305.89 &  116.25 & 116.25 &  4\,794.51 \\
                                          & Sym. & \xmark &            797.74 &  112.55 &  81.50 & 6\,137.12 &                    1\,727.17 &  245.81 & 245.81 & 17\,055.06 \\
                                          &      & \cmark &            270.95 &   53.85 &  33.75 & 1\,545.56 &                     649.91 &  108.87 & 108.87 

### Min key is Quatsch

In [14]:
queries.loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot', 'choose_direction_strategy'])[['running_time_ms', 'num_queue_pushs_k']].mean().unstack(0) \
    .loc[('probabilistic_scale_by_speed', 1.50)]

running_time_ms                \
potential                                                   ALT     CH Oracle   
probability bidir_pot choose_direction_strategy                                 
0.00        Average   alternating                        281.72 368.40 211.48   
                      min_key                            287.57 379.83 215.08   
            Symmetric alternating                        287.84 373.11 225.56   
                      min_key                            294.64 394.81 251.02   
1.00        Average   alternating                         19.06   8.97   4.31   
                      min_key                             31.21  16.39   8.05   
            Symmetric alternating                         37.11   9.88   5.06   
                      min_key                             55.28  19.04   9.95   

                                                       num_queue_pushs_k  \
potential                                         Zero               ALT   
probability bidir_pot choose_direction_strategy                            
0.00        Average   alternating               403.00            435.39   
                      min_key                   429.89            456.90   
            Symmetric alternating               400.60            443.59   
                      min_key                   422.61            473.22   
1.00        Average   alternating               384.59             33.44   
                      min_key                   417.73             52.66   
            Symmetric alternating               389.14             57.97   
                      min_key                   416.76             87.45   

                                                                        
potential                                           CH Oracle     Zero  
probability bidir_pot choose_direction_strategy                         
0.00        Average   alternating               403.96 403.96 1,057.22  
                      min_key                   418.52 418.52 1,118.97  
            Symmetric alternating               393.03 393.03 1,050.73  
                      min_key                   416.01 416.01 1,112.45  
1.00        Average   alternating                11.17  11.17   893.60  
                      min_key                    19.59  19.59   956.63  
            Symmetric alternating                 9.67   9.67   884.75  
                      min_key                    18.12  18.12   947.26

### Bidir Effectiveness

In [16]:
queries.loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot'])[['running_time_ms', 'num_queue_pushs_k']].mean().unstack(0)

running_time_ms  \
potential                                                             ALT   
experiment                   factor probability bidir_pot                   
probabilistic_scale_by_speed 1.25   0.00        Average            158.72   
                                                Symmetric          176.69   
                                                unidir             216.35   
                                    1.00        Average             17.21   
                                                Symmetric           37.35   
                                                unidir              75.84   
                             1.50   0.00        Average            281.72   
                                                Symmetric          287.84   
                                                unidir             334.79   
                                    1.00        Average             19.06   
                                                Symmetric           37.11   
                                                unidir              93.30   
weight_scale                 1.00   1.00        Average             15.07   
                                                Symmetric           42.85   
                                                unidir              43.37   
                             1.05   1.00        Average             34.53   
                                                Symmetric           75.87   
                                                unidir              95.95   
                             1.10   1.00        Average             63.15   
                                                Symmetric           99.61   
                                                unidir             130.45   

                                                                         \
potential                                                     CH Oracle   
experiment                   factor probability bidir_pot                 
probabilistic_scale_by_speed 1.25   0.00        Average   184.53 103.21   
                                                Symmetric 193.09 112.28   
                                                unidir    170.32 123.20   
                                    1.00        Average     4.50   2.01   
                                                Symmetric   5.07   2.35   
                                                unidir      7.18   4.61   
                             1.50   0.00        Average   368.40 211.48   
                                                Symmetric 373.11 225.56   
                                                unidir    318.50 228.56   
                                    1.00        Average     8.97   4.31   
                                                Symmetric   9.88   5.06   
                                                unidir     18.96  12.61   
weight_scale                 1.00   1.00        Average     0.83   0.19   
                                                Symmetric   0.71   0.20   
                                                unidir      0.48   0.17   
                             1.05   1.00        Average    17.85   8.97   
                                                Symmetric  19.65  10.37   
                                                unidir     16.16  10.74   
                             1.10   1.00        Average    52.29  27.97   
                                                Symmetric  57.01  31.18   
                                                unidir     56.70  39.33   

                                                                  \
potential                                                   Zero   
experiment                   factor probability bidir_pot          
probabilistic_scale_by_speed 1.25   0.00        Average   398.44   
                                                Symmetric 397.80   
                                                unidir    574.75   
